In [3]:
import numpy as np
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report ,confusion_matrix,accuracy_score
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, ReLU ,BatchNormalization, Dropout ,MaxPooling1D, Conv1D,Flatten,LSTM
from keras.optimizers import Adam
import pickle
from keras.preprocessing.text import Tokenizer

In [4]:
df=pd.read_parquet('/content/drive/MyDrive/silverline/MirrorMind/questionnaire/data_prepros.parquet')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 360237 entries, 0 to 51722
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   level         360237 non-null  object
 1   prepros_text  360237 non-null  object
dtypes: object(2)
memory usage: 8.2+ MB


In [5]:
df['level'].value_counts()

moderate    189695
mild        121103
minimal      26574
severe       22865
Name: level, dtype: int64

In [6]:
df_final=df[df['level']=='moderate'].iloc[:30000,]
df_final=df_final.append(df[df['level']=='mild'].iloc[:30000,])
print(df_final.shape)
df_final=df_final.append(df[df['level']=='minimal'])
print(df_final.shape)
df_final=df_final.append(df[df['level']=='severe'])
print(df_final.shape)

(60000, 2)
(86574, 2)
(109439, 2)


<ipython-input-6-3b4c19f9f22b>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final=df_final.append(df[df['level']=='mild'].iloc[:30000,])
<ipython-input-6-3b4c19f9f22b>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final=df_final.append(df[df['level']=='minimal'])
<ipython-input-6-3b4c19f9f22b>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final=df_final.append(df[df['level']=='severe'])


In [7]:
df_final['prepros_text'].iloc[0]

'would describe emotional state blue sad time cant snap sad unhappy quite painfulhow would describe overall outlook future feel nothing look forward towhat overall perception accomplishment failure life feel accomplished little worthwhile mean anything look back life see lot failureswhat level self worth believe currently posse feel quite guilty feel bad unworthy practically time nowhow would describe emotion considering possibility something bad happening feel punished punishedhow feel make mistake fail disgusted myselfthe extent current thought self harm suicide feel would better dead feel family would better deadwhat best describes current level irritation annoyance feel irritated time nowhow would rate ability approach making decision great difficulty making decisionshow feel current appearance change might noticed time feel permanent change appearance make look unattractivehow would describe current level motivation ability perform task push hard anythingwhat current level satisfa

In [8]:
len(df['prepros_text'].iloc[0].split())

114

In [9]:
checkpoint=ModelCheckpoint(
    '/content/drive/MyDrive/silverline/MirrorMind/questionnaire/model.h5',
    monitor='val_loss',
    mode='min',
    save_best_only=True,
)
stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    mode='min',
)

In [10]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_final['prepros_text'])
sequences = tokenizer.texts_to_sequences(df_final['prepros_text'])
X= pad_sequences(sequences, maxlen=150)



with open('/content/drive/MyDrive/silverline/MirrorMind/questionnaire/tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [11]:
X[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   2,   4,  27,  28, 164,
        26,  12,  66, 165,  26, 163, 109, 166,   2,   4,   8,  29,  16,
         1, 161,  61, 162,  86,   8,  30,  31,  25,   7,   1, 141, 142,
       143, 144,  73,  61, 145,   7, 146, 147, 148,   5,   9,  32,  33,
        34,  35,   1, 109, 139,   1,  11,  75, 140,  12,  74,   2,   4,
        36,  37,  38,  14,  11,  39,   1,  79,  63,   1,  13,  40,  41,
        94,  42,  43,   3,  18,   9,  44,  20,   1,   2,  69, 116,   1,
       117,   2,  69, 118,  45,  46,   3,   5,  47,  48,   1,  49,  12,
        74,   2,  50,  10,  51,  15,  17, 114, 115,  15,  70,   1,   3,
        22,  23,  52,  53,  12,   1, 113,  23,  22,  13,  61,  71,   2,
         4,   3,   5,  54,  10,  55,  56, 110, 111, 112,   3,   5,  21,
        57,  58,   7,   6,  19,  21,  73], dtype=int32)

In [12]:
label_encoder = LabelEncoder()
num=len(df_final['level'].unique())

encoded_labels = label_encoder.fit_transform(df_final['level'])
y = tf.keras.utils.to_categorical(encoded_labels, num_classes=num)
y

array([[0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.]], dtype=float32)

In [16]:
label_encoder.classes_

array(['mild', 'minimal', 'moderate', 'severe'], dtype=object)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
model = tf.keras.models.Sequential([
    Conv1D(128, kernel_size=2, input_shape=(X_train.shape[1], 1)),
    BatchNormalization(),
    ReLU(),
    MaxPooling1D(pool_size=2),
    Conv1D(256, kernel_size=3),
    BatchNormalization(),
    ReLU(),
    MaxPooling1D(pool_size=2),
    Conv1D(256, kernel_size=3),
    BatchNormalization(),
    ReLU(),
    MaxPooling1D(pool_size=2),
    LSTM(256,return_sequences=True),
    LSTM(512),
    Flatten(),
    Dense(512),
    BatchNormalization(),
    ReLU(),
    Dropout(0.5),
    Dense(256),
    BatchNormalization(),
    ReLU(),
    Dense(128),
    BatchNormalization(),
    ReLU(),
    Dropout(0.5),
    Dense(64),
    BatchNormalization(),
    ReLU(),
    Dense(num, activation='softmax')
])


model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_3 (Conv1D)           (None, 149, 128)          384       
                                                                 
 batch_normalization_7 (Batc  (None, 149, 128)         512       
 hNormalization)                                                 
                                                                 
 re_lu_7 (ReLU)              (None, 149, 128)          0         
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 74, 128)          0         
 1D)                                                             
                                                                 
 conv1d_4 (Conv1D)           (None, 72, 256)           98560     
                                                                 
 batch_normalization_8 (Batc  (None, 72, 256)         

In [ ]:
model.fit(X_train, y_train, epochs=100, batch_size=128,callbacks=[checkpoint,stop],validation_split=0.1)

Epoch 1/100
693/693 [==============================] - 29s 35ms/step - loss: 0.3269 - accuracy: 0.8950 - val_loss: 0.1831 - val_accuracy: 0.9422
Epoch 2/100
693/693 [==============================] - 21s 30ms/step - loss: 0.1313 - accuracy: 0.9550 - val_loss: 0.1097 - val_accuracy: 0.9560
Epoch 3/100
693/693 [==============================] - 22s 31ms/step - loss: 0.0868 - accuracy: 0.9695 - val_loss: 0.0625 - val_accuracy: 0.9752
Epoch 4/100
693/693 [==============================] - 20s 28ms/step - loss: 0.0714 - accuracy: 0.9745 - val_loss: 0.1176 - val_accuracy: 0.9545
Epoch 5/100
693/693 [==============================] - 20s 29ms/step - loss: 0.0588 - accuracy: 0.9778 - val_loss: 0.0463 - val_accuracy: 0.9790
Epoch 6/100
693/693 [==============================] - 21s 30ms/step - loss: 0.0510 - accuracy: 0.9812 - val_loss: 0.0238 - val_accuracy: 0.9904
Epoch 7/100
693/693 [==============================] - 19s 28ms/step - loss: 0.0461 - accuracy: 0.9830 - val_loss: 0.0807 - val_ac

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from keras.models import load_model

model=load_model('/content/drive/MyDrive/silverline/MirrorMind/questionnaire/model.h5')
result=model.predict(X_test)
print(classification_report(y_test.argmax(axis=1),result.argmax(axis=1)))

342/342 [==============================] - 4s 6ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3099
           1       1.00      0.99      1.00      2631
           2       1.00      1.00      1.00      3022
           3       1.00      1.00      1.00      2192

    accuracy                           1.00     10944
   macro avg       1.00      1.00      1.00     10944
weighted avg       1.00      1.00      1.00     10944
